On our journey to democratize finance for all, we’ve created the concept of fractional shares. Fractional shares are pieces, or fractions, of whole shares of a company or ETF.

However, exchanges only trade in whole shares. Robinhood is required to manage the fractional portion of each trade.

If Robinhood has 0 shares of AAPL and then a customer wishes to purchase 1.5 AAPL shares, Robinhood will need to request 2 shares from the exchange and hold on to the remaining 0.5 shares.
If another customer requests to purchase 0.4 shares of AAPL, Robinhood can use its inventory (0.5 shares) instead of going out to the exchange and will have 0.1 shares of AAPL remaining.
If the third customer requests 0.5 shares, Robinhood can fill 0.1 shares out of inventory but will need to go to the exchange for an additional share leaving Robinhood's inventory at 0.6 shares.
If a customer requests a dollar based order, we need to convert it to the relevant number of shares and run through the above steps.
Always ensure the firm has a positive quantity in inventory and has under one share after handling an order. There's no need for us to hold onto whole shares!
Steps:

Handle buying fractional shares.
Handle selling fractional shares.
Ensure inventory is less than 1 after each order.
e.g. Customer sells AAPL for 0.75 and then another sells AAPL for 0.50 -- we have 1.25 inventory. We can sell 1 share to the market and keep our inventory small at 0.25.
Ensure inventory is always non-negative after each order.
e.g. Inventory is 0.2 and the customer buys 0.5 shares: ensure we end up with 0.7 shares in inventory.
Always “flatten”! (steps 3+4)
The final 2 digits of every integer is the decimal. e.g. 1000 = 10.00, 20 = 0.20, 100 = 1.

Example scenario:

Input:
// One AAPL buy order for 0.42 shares. AAPL is currently worth $1.
orders: [["AAPL","B","42","100"]]

// Inventory for AAPL is currently 0.99 shares.
inventory: [["AAPL","99"]]


Expected Output:
// The users buys 0.42 shares from inventory, leaving us with 0.57 shares.
[["AAPL","57"]]
Another example scenario:

Input:
// One AAPL buy order for $0.42. AAPL is currently worth $1, so that's 0.42 shares.
orders: [["AAPL","B","$42","100"]]
// Existing AAPL inventory is 0.50 shares.
inventory: [["AAPL","50"]]

Expected Output:
// 0.50 - 0.42 = 0.08 shares leftover.
[["AAPL","8"]]

[execution time limit] 3 seconds (java)

[memory limit] 1 GB

[input] array.array.string orders

A list of orders in the format of [$SYMBOL, $BUY_OR_SELL, $QUANTITY, $CURRENT_PRICE]. Each parameter is a string.

$SYMBOL: Can be "AAPL", "GOOGL", "MEOOOOOW" or anything really.
$BUY_OR_SELL: "B" or "S". B for BUY, S for SELL.
$QUANTITY: Can be a number or a dollar amount (prefixed with $). e.g. "100" for 1 quantity or "$150" for $1.50.
$CURRENT_PRICE: Current price of the symbol with no $ sign. e.g. "1000" for $10.

** All numbers are multiplied by 100 to store two significant digits. e.g. 100 = 1.00, 150 = 1.50, 1025 = 10.25 **

[input] array.array.string inventory

Inventory is a list of the inventory of each symbol. Each element in the list a 2 item list of [$SYMBOL, $QUANTITY] (remember quantity is multiplied by 100!).

An example for AAPL of 0.50 shares and GOOGL of 0.75 shares would be:

[["AAPL","50"], 
 ["GOOG","75"]]
[output] array.array.string

The output is the final inventory of each symbol after iterating through each trade. This is expected to be in the same order and format as the inventory parameter.

e.g.

["AAPL","58"], 
 ["GOOG","50"]]

In [34]:
from collections import OrderedDict
class Stock:
    def __init__(self, symbol, inventory=0): # not storing price since it can change
        self.symbol = symbol
        self.inventory = inventory
        self.actions = [] #(is_buy, is_dollar, quantity, price)

    def buy_number(self, number):
        if number < self.inventory:
            self.inventory -= number
        else:
            number -= self.inventory
            request = number % 100
            self.inventory = 100 - request

    def buy_dollar(self, dollar, price): # real dollar, real price
        number = int(dollar / price * 100)
        self.buy_number(number)

    def sell_number(self, number):
        self.inventory = (self.inventory + number) % 100

    def sell_dollar(self, dollar, price):
        number = int(dollar / price * 100)
        self.sell_number(number)
    
def fractional_shares(orders, inventories):
    stocks = OrderedDict()
    for order in orders:
        symbol, action, quantity, price = order[0], order[1], order[2], int(order[3])
        if symbol not in stocks:
            stock = Stock(symbol)
            stocks[symbol] = stock
        if quantity.startswith("$"):
            dollar = int(quantity[1:])
            stocks[symbol].actions.append([action=="B", True, dollar, price])
        else:
            stocks[symbol].actions.append([action=="B", False, int(quantity), price])

    for inventory in inventories:
        symbol, quantity = inventory[0], int(inventory[1])
        stocks[symbol].inventory = quantity

    res = []
    for stock in stocks.values():
        for action in stock.actions:
            print(action)
            (is_buy, is_dollar, quantity, price) = action
            if is_buy and is_dollar:
                stock.buy_dollar(quantity, price)
            elif is_buy and not is_dollar:
                stock.buy_number(quantity)
            elif not is_buy and is_dollar:
                stock.sell_dollar(quantity, price)
            elif not is_buy and not is_dollar:
                stock.sell_number(quantity)
            print("stock={0}, inventory={1}".format(stock.symbol, stock.inventory))
        res.append([stock.symbol, str(stock.inventory)])
    return res

In [1]:
a = "20"
print(int(a)/100)

0.2


In [35]:
orders_0 = [["AAPL","B","42","100"], ["AAPL","B","$42","100"], ["AAPL","B","$60","100"]]
inventories_0 = [["AAPL","99"]]
fractional_shares(orders_0, inventories_0)

[True, False, 42, 100]
stock=AAPL, inventory=57
[True, True, 42, 100]
stock=AAPL, inventory=15
[True, True, 60, 100]
stock=AAPL, inventory=55


[['AAPL', '55']]

In [36]:
# If Robinhood has 0 shares of AAPL and then a customer wishes to purchase 1.5 AAPL shares, Robinhood will need to request 2 shares from the exchange and hold on to the remaining 0.5 shares.
# If another customer requests to purchase 0.4 shares of AAPL, Robinhood can use its inventory (0.5 shares) instead of going out to the exchange and will have 0.1 shares of AAPL remaining.
# If the third customer requests 0.5 shares, Robinhood can fill 0.1 shares out of inventory but will need to go to the exchange for an additional share leaving Robinhood's inventory at 0.6 shares.
orders_1 = [["AAPL","B","150","100"],["AAPL","B","40","100"],["AAPL","B","50","100"]]
inventories_1 = [["AAPL","0"]]
expected = [50, 10, 60]
fractional_shares(orders_1, inventories_1)

[True, False, 150, 100]
stock=AAPL, inventory=50
[True, False, 40, 100]
stock=AAPL, inventory=10
[True, False, 50, 100]
stock=AAPL, inventory=60


[['AAPL', '60']]

In [37]:
# e.g. Customer sells AAPL for 0.75 and then another sells AAPL for 0.50 -- we have 1.25 inventory. We can sell 1 share to the market and keep our inventory small at 0.25.
# Ensure inventory is always non-negative after each order.
# e.g. Inventory is 0.2 and the customer buys 0.5 shares: ensure we end up with 0.7 shares in inventory.
orders_2 = [["AAPL","S","75","100"],["AAPL","S","50","100"],["AAPL","B","5","100"]]
inventories_2 = [["AAPL","0"]]
expected = [75, 25, 20]
fractional_shares(orders_2, inventories_2)

[False, False, 75, 100]
stock=AAPL, inventory=75
[False, False, 50, 100]
stock=AAPL, inventory=25
[True, False, 5, 100]
stock=AAPL, inventory=20


[['AAPL', '20']]